In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys

sys.path.append("../src")

In [ ]:
import pandas as pd
from db_building import reload_database_from_local_dir
from evaluation import RetrievalConfiguration, RetrievalEvaluator

In [ ]:
path = "./data/q_and_a_ref_retrieval_evaluation_Phi-3-mini-128k-instruct.csv"
df = pd.read_csv(path)

df_dict = {}
df_dict["the_df_dataset"] = df

In [ ]:
db = reload_database_from_local_dir()

In [ ]:
eval_configs = [RetrievalConfiguration("conf_1"), RetrievalConfiguration("conf_2")]

d = RetrievalEvaluator.run(
    eval_configs,
    db,
    df_dict,
)

In [ ]:
d

In [ ]:
////////////////////////////////////////////////////////////////////////////